In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

# restore saved variables
%store -r summary_felony6_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_six_month'].values

In [4]:
#### CART
depth = [7,8,9]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [60]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [6]:
cart_summary['FL_score'],  ebm_summary['FL_score']

(0.5595231176546558, 0.5350833010461061)

### Lasso Stumps

In [7]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_six_month'].values

## columns 
cols = KY_X.columns

In [8]:
c_grid = {'C':[0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary

{'best_auc': 0.8059820707120038,
 'best_params': {'C': 0.002},
 'number of coefs': 34,
 'dictionary': {'age_at_current_charge19.0': 0.049038721270004346,
  'age_at_current_charge21.0': 0.025208590497949015,
  'age_at_current_charge22.0': 0.0012948361342319731,
  'age_at_current_charge26.0': 0.01397404663929515,
  'age_at_current_charge31.0': 0.031160293731802756,
  'age_at_current_charge35.0': 0.06511883288176985,
  'age_at_current_charge36.0': 0.03653858301579923,
  'age_at_current_charge39.0': 0.03258456687215333,
  'age_at_current_charge41.0': 0.044003587273430456,
  'age_at_current_charge45.0': 0.11610825975602417,
  'age_at_current_charge70.0': -0.4940579233283735,
  'p_arrest1': -0.5436052750783779,
  'p_arrest2': 0.6023432474949036,
  'p_arrest3': 0.2716470326309807,
  'p_arrest4': 0.20380315167654045,
  'p_arrest5': 0.07182188786633352,
  'p_arrest6': 0.08810268392237994,
  'p_charges1': -0.3631786098908708,
  'p_charges2': 0.0650020961583165,
  'p_violence1': 0.225866373223091

### RiskSLIM

In [10]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_six_month'].values

## columns 
cols = KY_X.columns

In [11]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.001, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

18

In [12]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [13]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'felony_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=2000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/22/19 @ 01:43 PM | 233 rows in lookup table
10/22/19 @ 01:43 PM | ------------------------------------------------------------
10/22/19 @ 01:43 PM | runnning initialization procedure
10/22/19 @ 01:43 PM | ------------------------------------------------------------
10/22/19 @ 01:43 PM | CPA produced 2 cuts
10/22/19 @ 01:43 PM | running naive rounding on 6 solutions
10/22/19 @ 01:43 PM | best objective value: 0.4933
10/22/19 @ 01:43 PM | rounding produced 5 integer solutions
10/22/19 @ 01:43 PM | best objective value is 0.5167
10/22/19 @ 01:43 PM | running sequential rounding on 6 solutions
10/22/19 @ 01:43 PM | best objective value: 0.4933
10/22/19 @ 01:43 PM | sequential rounding produced 4 integer solutions
10/22/19 @ 01:43 PM | best objective value: 0.3481
10/22/19 @ 01:43 PM | polishing 9 solutions
10/22/19 @ 01:43 PM | best objective value: 0.3481
10/22/19 @ 01:43 PM | polishing produced 5 integer solutions
10/22/19 @ 0

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 01:43 PM | adding 255 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1903.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 145437 54480        0.1778     7        0.1829        0.1748   749654    4.41%           rho_3 D 145437 145436     32
 148266 55428        0.1787    10        0.1829        0.1749   762402    4.38%          rho_10 D 148266 148264     34
Elapsed time = 47.03 sec. (41259.35 ticks, tree = 15.95 MB, solutions = 8)
 151077 56338        0.1825     2        0.1829        0.1749   775287    4.35%           rho_2 U 151077 151075     33
 154166 57737        0.1793     7        0.1829        0.1749   787234    4.35%           rho_2 D 154166 154165     36
 157060 58891        0.1819     5        0.1829        0.1749   798777    4.33%          rho_17 D 157060 157059     36
 159840 59754        0.1763    13        0.1829        0.1750   811132    4.31%          rho_14 D 159840 159839     34
 162648 60699        cutoff              0.1829        0.1750   823476    4.28%           rho_5 U 162648  72363     27
 165614 61870        cutoff              0.1829        0.1751   835388    4.27%           rh

 425198 126403        0.1781    12        0.1820        0.1769  1870003    2.79%           rho_4 U 425198 425197     37
 434760 128637        0.1805     7        0.1820        0.1769  1913520    2.76%           rho_2 U 434760 434758     42
 445405 131029        0.1803     7        0.1820        0.1770  1952385    2.73%         alpha_7 D 445405 445404     32
 456863 134781        0.1786    16        0.1820        0.1770  1991908    2.71%          rho_10 D 456863 402631     28
 467390 135577        cutoff              0.1820        0.1771  2031683    2.69%           rho_1 U 467390 467387     28
 477885 137317        0.1782     9        0.1820        0.1771  2070549    2.67%           rho_2 D 477885 477884     36
 489151 140500        0.1805     9        0.1820        0.1771  2110178    2.65%          rho_14 U 489151 489149     30
 500460 143665        0.1811     9        0.1820        0.1772  2151927    2.64%           rho_2 U 500460 500459     45
 511525 146555        cutoff            

 1060650 202113        0.1800    10        0.1820        0.1790  4212248    1.63%           rho_3 U 1060650 1060649     32
 1071077 203812        0.1804     9        0.1820        0.1790  4244547    1.62%           rho_2 D 1071077 1071075     39
 1080664 203637        0.1812     9        0.1820        0.1790  4277736    1.61%           rho_4 N 1080664 1080663     41
 1090312 203530        0.1805     3        0.1820        0.1791  4311531    1.59%        alpha_16 D 1090312 1090311     37
 1100135 204126        0.1804     9        0.1820        0.1791  4346178    1.57%           rho_1 U 1100135  45740     28
 1109841 205090        0.1799     9        0.1820        0.1791  4380811    1.57%           rho_2 D 1109841 1109839     46
Elapsed time = 349.09 sec. (397017.58 ticks, tree = 66.04 MB, solutions = 10)
 1119420 205721        0.1799     8        0.1820        0.1791  4412541    1.55%           rho_2 D 1119420 1119419     35
 1128750 204441        0.1796     9        0.1820        0.179

...

10/22/19 @ 02:24 PM | ------------------------------------------------------------
10/22/19 @ 02:24 PM | CPA produced 2 cuts
10/22/19 @ 02:24 PM | running naive rounding on 6 solutions
10/22/19 @ 02:24 PM | best objective value: 0.4909
10/22/19 @ 02:24 PM | rounding produced 5 integer solutions
10/22/19 @ 02:24 PM | best objective value is 0.5630
10/22/19 @ 02:24 PM | running sequential rounding on 6 solutions
10/22/19 @ 02:24 PM | best objective value: 0.4909
10/22/19 @ 02:24 PM | sequential rounding produced 4 integer solutions
10/22/19 @ 02:24 PM | best objective value: 0.3473
10/22/19 @ 02:24 PM | polishing 9 solutions
10/22/19 @ 02:24 PM | best objective value: 0.3473
10/22/19 @ 02:24 PM | polishing produced 5 integer solutions
10/22/19 @ 02:24 PM | initialization produced 14 feasible solutions
10/22/19 @ 02:24 PM | best objective value: 0.1899
10/22/19 @ 02:24 PM | ------------------------------------------------------------
10/22/19 @ 02:24 PM | completed initialization procedur

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/22/19 @ 02:24 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.1899.
Tried aggregator 1 time.
Reduced MIP has 38 rows, 40 columns, and 111 nonzeros.
Reduced MIP has 18 binaries, 20 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 146384 58593        cutoff              0.1816        0.1749   832489    3.69%           rho_5 U 146384 146382     22
 149332 59618        cutoff              0.1816        0.1749   846224    3.66%          rho_18 D 149332 149331     31
Elapsed time = 83.73 sec. (41257.80 ticks, tree = 16.41 MB, solutions = 9)
 152290 60630        0.1813    11        0.1816        0.1750   860689    3.63%        alpha_16 D 152290 152289     25
 155100 61435        0.1779    10        0.1816        0.1750   875127    3.61%          rho_12 U 155100 155099     31
 158244 62886        0.1764    14        0.1816        0.1750   889951    3.60%           rho_3 U 158244 158243     37
 161292 64209        0.1759    12        0.1816        0.1751   904472    3.59%          rho_12 U 161292 161291     33
 164293 65242        0.1813     3        0.1816        0.1751   918677    3.57%          rho_10 D 164293 137854     21
 167250 66222        0.1765    16        0.1816        0.1751   932936    3.55%           rh

 450273 138319        cutoff              0.1816        0.1775  2218029    2.26%          rho_14 D 450273  90629     20
 461172 139368        0.1797     8        0.1816        0.1776  2268204    2.22%         alpha_9 D 461172 461171     30
 471990 140435        0.1794    11        0.1816        0.1776  2314828    2.17%          rho_14 D 471990 215185     28
 483471 142723        0.1781     9        0.1816        0.1777  2363734    2.13%          rho_12 U 483471 206152     27
 494659 144412        cutoff              0.1816        0.1778  2409686    2.09%         alpha_7 U 494659 494657     30
 505687 146268        0.1778    12        0.1816        0.1778  2454755    2.06%           rho_5 U 505687 505617     32
 517093 148585        cutoff              0.1816        0.1779  2500921    2.03%          rho_14 D 517093 191293     18
 528049 150341        0.1779    13        0.1816        0.1779  2546157    2.00%           rho_0 U 528049 528032     27
Elapsed time = 285.14 sec. (168114.30 ti

 1100846 67885        cutoff              0.1813        0.1804  4674920    0.46%         alpha_8 U 1100846 302877     37
 1110754 62227        cutoff              0.1813        0.1805  4700175    0.42%          rho_18 U 1110754 315148     29
 1119947 55657        cutoff              0.1813        0.1806  4724265    0.37%          rho_18 U 1119947 409626     30
 1130018 50771        cutoff              0.1813        0.1807  4747656    0.34%         alpha_8 U 1130018 1033306     27
 1139187 43946        cutoff              0.1813        0.1807  4768861    0.30%         alpha_7 U 1139187 201979     30
Elapsed time = 627.00 sec. (397009.42 ticks, tree = 21.13 MB, solutions = 10)
 1148696 37603        cutoff              0.1813        0.1808  4789527    0.26%           rho_5 D 1148696 948718     30
 1158534 31966        0.1810    10        0.1813        0.1809  4810704    0.22%           rho_5 N 1158534 308811     29
 1168052 25345        cutoff              0.1813        0.1810  4829613   

In [15]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_train_score'])

(0.5476979207025701, 0.7778887467171796)

In [16]:
#### save results
summary_felony6_ky_inter_model = {"CART": cart_summary,
                                  "EBM": ebm_summary,
                                  "Stumps": stumps_summary,
                                  "RiskSLIM": riskslim_summary}
%store summary_felony6_ky_inter_model

Stored 'summary_felony6_ky_inter_model' (dict)


In [17]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["Felony",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)